<h1 style="text-align: center; font-weight: bold;">Data Sciense Course - Spring 1403</h1>
<h1 style="text-align: center; font-weight: bold;">Project Phase1 - Introduction to Data Science </h1>
<h1 style="text-align: center;">Mohammadreza Mohammadhashemi : 810100206</h1>
<h1 style="text-align: center;">Soheil Hajian Manesh : 810100119</h1>
<h1 style="text-align: center;">Mahdi Ebrahimi Soltani : 810100241</h1>


# Problem Description
in this phase we are going to scrapping datas from fiverr.com website using BeautifulSoup and selenium packages and then extract some meaningful statistics from dataset using EDA and visualisation and preprocessing steps.

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import os
from fiverr_api import session

## Data Collection

In [3]:
def initialize_driver(proxy=None):
    options = Options()
    options.add_argument(r"user-data-dir=C:\Users\Lenovo\AppData\Local\Google\Chrome\User Data")
    options.add_argument(r"profile-directory=Profile 1")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")
    driver = webdriver.Chrome(options=options)
    return driver

base_url = "https://www.fiverr.com"

In [4]:
def get_text_or_default(element, default="N/A"):
    return element.text.strip() if element else default

In [5]:
def extract_price_info(table):
    prices = table.find_all("div", class_="price-wrapper")
    return (
        get_text_or_default(prices[0]),
        get_text_or_default(prices[1]),
        get_text_or_default(prices[2]),
    )

In [6]:
def extract_price_info_single_plan(soup):
    price = soup.find("div", class_="price-wrapper")
    return (
        get_text_or_default(price),
        get_text_or_default(price),
        get_text_or_default(price)
    )

In [7]:
def extract_delivery_info(table):
    row = table.find("tr", class_="delivery-time")
    tds = row.find_all("td")
    delivery_0 = tds[1].find_all("label")[0] if len(tds[1].find_all("label"))>0 else tds[1]
    delivery_1 = tds[2].find_all("label")[0] if len(tds[2].find_all("label"))>0 else tds[2]
    delivery_2 = tds[3].find_all("label")[0] if len(tds[3].find_all("label"))>0 else tds[3]
    return (
            get_text_or_default(delivery_0),
            get_text_or_default(delivery_1),
            get_text_or_default(delivery_2),
        )

In [8]:
def extract_delivery_info_single_plan(soup):
    delivery = soup.find_all("div", class_="delivery-wrapper")[0]
    return (
        get_text_or_default(delivery),
        get_text_or_default(delivery),
        get_text_or_default(delivery),
    )

In [9]:
def extract_revision_info(table):
    rows = table.find_all("tr")
    for row in rows[1:]:
        if row.find_all("td")[0].text.strip() == "Revisions":
            revision_row = row
            break

    revisions = revision_row.find_all("td") 
    return (
        get_text_or_default(revisions[1]),
        get_text_or_default(revisions[2]),
        get_text_or_default(revisions[3]),
    )

In [10]:
def extract_revision_info_single_plan(soup):
    revision = soup.find_all("div", class_="revisions-wrapper")[0]
    return (
        get_text_or_default(revision),
        get_text_or_default(revision),
        get_text_or_default(revision),
    )

In [11]:
def extract_user_stats(soup):
    user_stats = soup.find("ul", class_="user-stats").find_all("li")
    return (
        get_text_or_default(user_stats[0].find("strong")),
        get_text_or_default(user_stats[1].find("strong")),
        get_text_or_default(user_stats[2].find("strong")),
        get_text_or_default(user_stats[3].find("strong")),
        get_text_or_default(user_stats[4].find("strong")),
    )

In [12]:
def extract_rating_info(soup):
    rating = get_text_or_default(soup.find("b", class_="rating-score"))
    rating_count = get_text_or_default(soup.find("span", class_="rating-count-number"))
    return rating, rating_count

In [13]:
def extract_order_in_queue(soup):
    order_in_queue = soup.find("div", class_="FVVUrQM")
    return get_text_or_default(order_in_queue)

In [19]:
def extract_records(driver, url, category, field, level, sellers_in_same_level):
    try:
        driver.get(url)

        body = driver.find_element(By.CLASS_NAME, "layout-service")

        html_content = body.get_attribute("innerHTML")

        soup = BeautifulSoup(html_content, "html.parser")

        table = soup.find("div", class_="gig-page-packages-table")

        if table:
            is_single_plan = False
            basic_price, standard_price, premium_price = extract_price_info(table)
            basic_delivery, standard_delivery, premium_delivery = extract_delivery_info(table)
            basic_revision, standard_revision, premium_revision = extract_revision_info(table)
        else:
            is_single_plan = True
            basic_price, standard_price, premium_price = extract_price_info_single_plan(soup)
            basic_delivery, standard_delivery, premium_delivery = extract_delivery_info_single_plan(soup)
            basic_revision, standard_revision, premium_revision = extract_revision_info_single_plan(soup)
        rating, rating_count = extract_rating_info(soup)

        country, member_since, avg_response_time, last_delivery, language = extract_user_stats(soup)

        order_in_queue = extract_order_in_queue(soup)

        return (
                category, field, level, sellers_in_same_level,
                basic_price, standard_price, premium_price,
                basic_delivery, standard_delivery, premium_delivery,
                basic_revision, standard_revision, premium_revision,
                rating, rating_count, country, member_since,
                avg_response_time, last_delivery, language, order_in_queue,
                is_single_plan
            )
    except Exception as e:
        return None

In [21]:
# category_fields_mapping = {
#     "Graphics & Design": {
#         "/categories/graphics-design/game-art",
#         "/categories/graphics-design/social-media-design",
#         "/categories/graphics-design/fashion-design",
#     },
#     "Music & Audio": {
#         "/categories/music-audio/mixing-mastering",
#         "/categories/music-audio/producers",
#         "/categories/music-audio/singers-vocalists",
#         "/categories/music-audio/songwriters",
#     },
#     "Programming & Tech": {
#         "/categories/programming-tech/website-development",
#         "/categories/programming-tech/software-development",
#         "/categories/programming-tech/mobile-app-services",
#         "/categories/programming-tech/game-development",
#     },
#     "Digital Marketing": {
#         "/categories/online-marketing/social-marketing",
#         "/categories/online-marketing/seo-services",
#         "/categories/online-marketing/online-video-marketing",
#     },
#     "Business": {
#         "/categories/business/financial-consulting-services",
#         "/categories/business/sales",
#         "/categories/business/business-plans",
#         "/categories/business/software-management",
#     },
#     "Writing & Translation": {
#         "/categories/writing-translation/articles-blogposts",
#         "/categories/writing-translation/quality-translation-services",
#         "/categories/writing-translation/book-editing",
#     },
#     "Data": {
#         "/categories/data/data-mining",
#         "/categories/data/databases",
#         "/categories/data/data-engineering",
#         "/categories/data/data-science",
#     },
#     "Photography": {
#         "/categories/photography/product-photographers",
#         "/categories/photography/event-photographers",
#         "/categories/photography/real-estate-photographers",
#     },
#     "Lifestyle": {
#         "/categories/lifestyle/online-tutoring",
#         "/categories/lifestyle/personal-stylists",
#         "/categories/lifestyle/fitness",
#         "/categories/lifestyle/modeling-acting",
#     },
#     "Video & Animation": {
#         "/categories/video-animation/video-editing",
#         "/categories/video-animation/visual-effects",
#         "/categories/video-animation/animated-explainer-videos",
#     },
# }

In [22]:
columns = [
    "Category",
    "Field",
    "Seller Level",
    "Seller In Same Level",
    "Basic Price",
    "Standard Price",
    "Premium Price",
    "Basic Delivery",
    "Standard Delivery",
    "Premium Delivery",
    "Basic Revision",
    "Standard Revision",
    "Premium Revision",
    "Rating",
    "Rating Count",
    "Country",
    "Member Since",
    "Avg Response Time",
    "Last Delivery",
    "Language",
    "Order in Queue",
    "is_single_plan",
]

In [23]:
def process_html_files(base_path):
    error_links = []
    driver = initialize_driver()
    session.set_scraper_api_key("cb672d07ab5658311e8958cc0dfffe13")
    temp_file_path = 'temp.html'
    for category in os.listdir(base_path):
        records = []
        category_path = os.path.join(base_path, category)
        if os.path.isdir(category_path):
            for field in os.listdir(category_path):
                field_path = os.path.join(category_path, field)
                if os.path.isdir(field_path):
                    for file_name in ["0.html", "1.html", "2.html", "3.html"]:
                        file_path = os.path.join(field_path, file_name)
                        if os.path.isfile(file_path):
                            with open(file_path, "r", encoding="utf-8") as file:
                                html_content = file.read()
                            soup = BeautifulSoup(html_content, "html.parser")
                            item_divs = soup.find_all("div", class_="gig-card-layout")
                            sellers_in_same_level = get_text_or_default(soup.find("p", class_="number-of-results"))
                            item_links = []
                            for div in item_divs:
                                link = div.find_all("a", href=True)[2]
                                if link:
                                    item_links.append(
                                        "https://www.fiverr.com" + link["href"]
                                    )
                            seller_level = [
                                "top rated seller",
                                "level 1",
                                "level 2",
                                "new seller",
                            ][int(file_name[0])]
                            for link in item_links:
                                response = session.get(link)
                                with open(temp_file_path, 'w', encoding='utf-8') as file:
                                    file.write(str(response.soup))
                                page_url = os.path.abspath(temp_file_path)
                                record = extract_records(driver, page_url, category, field, seller_level,sellers_in_same_level)
                                if record:
                                    records.append(record)
                                else:
                                    error_links.append(link)
        category_df = pd.DataFrame(records, columns=columns)
        os.makedirs("CSVs", exist_ok=True)
        csv_file_path = os.path.join("CSVs", f"{category}.csv")
        category_df.to_csv(csv_file_path, index=False)

    driver.quit()
    return records

In [ ]:
base_path = "htmls"
process_html_files(base_path)